<a id="overview"></a>

# Overview

The motivation and the data used for this project came from the [Titanic - Machine Learning from Disaster](https://www.kaggle.com/c/titanic/overview) competition from Kaggle.

In [ ]:
! pip install scikit-learn

In [1]:
import pandas as pd

from sklearn.ensemble import RandomForestClassifier

In [18]:
train_data = pd.read_csv("./data/train.csv")
test_data = pd.read_csv("./data/test.csv")

features = ["Pclass", "Sex", "SibSp", "Parch"]

train_x = pd.get_dummies(train_data[features])
train_y = train_data['Survived']
test_x = pd.get_dummies(test_data[features])

model = RandomForestClassifier(n_estimators=100, max_depth=5, random_state=1)
model.fit(train_x, train_y)
predictions = model.predict(test_x)

res = pd.DataFrame({"PassengerId": test_data['PassengerId'], "Survived": predictions})
res.to_csv("Titanic Submission.csv", index = False)

Index(['PassengerId', 'Pclass', 'Name', 'Sex', 'Age', 'SibSp', 'Parch',
       'Ticket', 'Fare', 'Cabin', 'Embarked'],
      dtype='object')
